In [1]:
import matplotlib.pyplot as plt
import json
import numpy as np
from typing import List, Dict
from matplotlib.lines import Line2D 

In [2]:

import numpy as np
from typing import List, Dict, Tuple

def aggregate_stats(
    data: List[List[Dict[str, Dict[str, float]]]]
) -> List[Dict[str, Dict[str, Tuple[float, float, float]]]]:
    if not data:
        return []

    num_items = len(data[0])
    result = []

    for i in range(num_items):
        accumulator: Dict[str, Dict[str, List[float]]] = {}

        for seed in data:
            entry = seed[i]
            for key1, subdict in entry.items():
                if key1=="accuracy":
                    continue
                if key1 not in accumulator:
                    accumulator[key1] = {}
                for key2, value in subdict.items():
                    accumulator[key1].setdefault(key2, []).append(value)

        # Compute (max, mean, std) for each float list
        stats_item: Dict[str, Dict[str, Tuple[float, float, float]]] = {}
        for key1, subdict in accumulator.items():
            stats_item[key1] = {}
            for key2, values in subdict.items():
                arr = np.array(values)
                max_val = float(np.max(arr))
                mean_val = float(np.mean(arr))
                std_val = float(np.std(arr, ddof=1)) if len(arr) > 1 else 0.0
                stats_item[key1][key2] = (max_val, mean_val, std_val)

        result.append(stats_item)

    return result


In [3]:
Models=["Rotation","L1HS16-24","L5HS16-24","L10HS16-24","L10HS128"]
layers = ["Layer1", "Layer2", "Layer3"]
EpochNames=["Random Init",
            "EP1-B131072",
            "EP1-B262144",
            "EP1-B393216",
            "EP1-B524288",
            "EP1-B655360",
            "EP1-B786432",
            "EP1-B917504",
            "EP1-B1048576",
            "EP2-B131072",
            "EP2-B262144",
            "EP2-B393216",
            "EP2-B524288",
            "EP2-B655360",
            "EP2-B786432",
            "EP2-B917504",
            "EP2-B1048576",]
def Prepare_Data(TaskName,Model_Names,layers_label,intervention_sizes,intervention_label):
    residict={}
    for ac_model_pos,ac_Model in enumerate(Models):
        with open('../TrainingProgression/'+ac_Model+'/'+TaskName+'/results.json') as f:
            results= json.load(f)
        results=aggregate_stats(results)

        for ac_train_step,ac_train_list in enumerate(results):
            if EpochNames[ac_train_step] not in residict:
                residict[EpochNames[ac_train_step]]={}
            for aclayer_pos,aclayer in enumerate(layers):
                if layers_label[aclayer_pos] not in residict[EpochNames[ac_train_step]]:
                    residict[EpochNames[ac_train_step]][layers_label[aclayer_pos]]={}
                for acint_pos, acint in enumerate(intervention_sizes):
                    if intervention_label[acint_pos] not in residict[EpochNames[ac_train_step]][layers_label[aclayer_pos]]:
                        residict[EpochNames[ac_train_step]][layers_label[aclayer_pos]][intervention_label[acint_pos]]=[[],[],[]]

                    residict[EpochNames[ac_train_step]][layers_label[aclayer_pos]][intervention_label[acint_pos]][0].append(results[ac_train_step][aclayer][acint][0])
                    residict[EpochNames[ac_train_step]][layers_label[aclayer_pos]][intervention_label[acint_pos]][1].append(results[ac_train_step][aclayer][acint][1])
                    residict[EpochNames[ac_train_step]][layers_label[aclayer_pos]][intervention_label[acint_pos]][2].append(results[ac_train_step][aclayer][acint][2])
    return residict


In [4]:
colors=[]
start_color=(52, 225, 235)
end_color=(5, 7, 66)
#start_color=(0, 255, 0)
#end_color=(255, 0, 0)
training_steps=len(EpochNames)
for i in range(training_steps):
    fac=i/(training_steps-1)
    R=start_color[0]+fac*(end_color[0]-start_color[0])
    G=start_color[1]+fac*(end_color[1]-start_color[1])
    B=start_color[2]+fac*(end_color[2]-start_color[2])
    colors.append((R/255,G/255,B/255))

In [5]:
import matplotlib.pyplot as plt

def make_plot(results,layers_label,intervention_label,url,mode=0):
    sub_y_min = 0.4
    sub_y_max = 1
    meancolor = (0/255, 128/255, 255/255)
    std_color = (52/255, 225/255, 235/255)
    rotationmean_color = (0/255, 153/255, 0/255)
    # Create a 3x3 grid for subplots
    fig, axes = plt.subplots(3, 3, figsize=(12, 8))
    
    # Iterate over the plot_data dictionary to create subplots
    for i, layer in enumerate(layers_label):
        for j, label in enumerate(intervention_label):
            ax = axes[i, j]
            
            
            
            # Plot the mean values
            for ac_step,step_la in enumerate(EpochNames):
                y_values_max = results[step_la][layer][label][0]
                y_values_mean = results[step_la][layer][label][1]
                y_values_std = results[step_la][layer][label][2]
                
                
                if mode==0:   
                    ax.plot(Models, y_values_mean, marker='o', color=colors[ac_step])
                    """
                    upper_bound = [m + s for m, s in zip(y_values_mean, y_values_std)]
                    lower_bound = [m - s for m, s in zip(y_values_mean, y_values_std)]
                    ax.fill_between(Models, lower_bound, upper_bound, 
                        color=colors[ac_step], alpha=0.25, linewidth=0)
                    """
                if mode==1:   
                    ax.plot(Models, y_values_max, marker='o', color=colors[ac_step])
                    """
                    upper_bound = [m + s for m, s in zip(y_values_mean, y_values_std)]
                    lower_bound = [m - s for m, s in zip(y_values_mean, y_values_std)]
                    ax.fill_between(Models, lower_bound, upper_bound, 
                        color=colors[ac_step], alpha=0.25, linewidth=0)
                    """
                        
            
 
            
            ax.set_ylim(sub_y_min, sub_y_max)
            
            if i != 2:
                ax.set_xlabel('')
                ax.set_xticks([])
            else:
                ax.set_xlabel(label)
                
            if j != 0:
                ax.set_ylabel('')
                ax.set_yticks([])
            else:
                ax.set_ylabel(layer)
        
            plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
            
            #plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
    
    # --- Add custom legend ---
    # Create custom legend handles
    legend_elements = [Line2D([0], [0], marker='o', color='w', label=EpochNames[k],
                              markerfacecolor=colors[k], markersize=10)
                       for k in range(len(EpochNames))]
    
    # Place legend outside the plot
    fig.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(0.85, 0.5))
    
    # Adjust layout
    plt.tight_layout(rect=[0.08, 0.08, 0.85, 0.92])  # make space for legend on right

    # Save the plot
    plt.savefig(url, bbox_inches='tight')
    plt.close()


## Plot for Both Equality Relations

In [6]:
intervention_sizes =   ['[[0], [1]]',
                        '[[0, 1], [2, 3]]',
                        '[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]]'
                        ]
layers_label=["Layer 1","Layer 2","Layer 3"]
intervention_label=["Intervetion Size 1","Intervetion Size 2","Intervetion Size 8"]
Model_Names=["Rotation","RevNet L1HS16","RevNet L5HS16","RevNet L10HS16","RevNet L10HS128"]
TaskName="Both_Equality_Relations"
results=Prepare_Data(TaskName,Model_Names,layers_label,intervention_sizes,intervention_label)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Mean.png",mode=0)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Max.png",mode=1)

## Plot for Left Equality Relation

In [7]:
intervention_sizes =   ['[[0], [1]]',
                        '[[0, 1], [2, 3]]',
                        '[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]]'
                        ]
layers_label=["Layer 1","Layer 2","Layer 3"]
intervention_label=["Intervetion Size 1","Intervetion Size 2","Intervetion Size 8"]
Model_Names=["Rotation","RevNet L1HS16","RevNet L5HS16","RevNet L10HS16","RevNet L10HS128"]
TaskName="Left_Equality_Relation"
results=Prepare_Data(TaskName,Model_Names,layers_label,intervention_sizes,intervention_label)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Mean.png",mode=0)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Max.png",mode=1)

## Plot for Identity of First Argument

In [8]:
intervention_sizes =   ['[[0]]',
                        '[[0, 1]]',
                        '[[0, 1, 2, 3, 4, 5, 6, 7]]'
                        ]
layers_label=["Layer 1","Layer 2","Layer 3"]
intervention_label=["Intervetion Size 1","Intervetion Size 2","Intervetion Size 8"]
Model_Names=["Rotation","RevNet L1HS16","RevNet L5HS16","RevNet L10HS16","RevNet L10HS128"]
TaskName="Identity_of_First_Argument"
results=Prepare_Data(TaskName,Model_Names,layers_label,intervention_sizes,intervention_label)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Mean.png",mode=0)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Max.png",mode=1)

## Plot for AndOrAnd

In [9]:
intervention_sizes =   ['[[0], [1]]',
                        '[[0, 1], [2, 3]]',
                        '[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]'
                        ]
layers_label=["Layer 1","Layer 2","Layer 3"]
intervention_label=["Intervetion Size 1","Intervetion Size 2","Intervetion Size 8"]
Model_Names=["Rotation","RevNet L1HS16","RevNet L5HS16","RevNet L10HS16","RevNet L10HS128"]
TaskName="AndOrAnd"
results=Prepare_Data(TaskName,Model_Names,layers_label,intervention_sizes,intervention_label)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Mean.png",mode=0)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Max.png",mode=1)

## Plot for AndOrAnd

In [10]:
intervention_sizes =   ['[[0], [1]]',
                        '[[0, 1], [2, 3]]',
                        '[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]'
                        ]
layers_label=["Layer 1","Layer 2","Layer 3"]
intervention_label=["Intervetion Size 1","Intervetion Size 2","Intervetion Size 8"]
Model_Names=["Rotation","RevNet L1HS16","RevNet L5HS16","RevNet L10HS16","RevNet L10HS128"]
TaskName="AndOr"
results=Prepare_Data(TaskName,Model_Names,layers_label,intervention_sizes,intervention_label)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Mean.png",mode=0)
make_plot(results,layers_label,intervention_label,"./Training_Progression_Plots/Training_Progression"+TaskName+"_Max.png",mode=1)